# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [2]:
import getpass
import pyspark
from datetime import datetime
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from pyspark.sql.types import BooleanType
from pyspark.sql.window import Window
import math
import helpers
import pickle

%load_ext autoreload
%autoreload 2

## Initialize the `SparkSession`

In [3]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '6g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-rbsteinm>

In [4]:
# init spark session
spark = SparkSession(sc)

## Loading the data

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe:

In [5]:
# load metadata
raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

In [6]:
# remove multiple spaces
metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
# split into columns
metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
    metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
# remove useless column
metadata = metadata.drop('_c0')
# trim name column to remove left/right blank
metadata = metadata.withColumn('name', functions.trim(metadata.name))

In [7]:
metadata.show(5)

+----------------+----------+---------+---------+------+
|            name|station_ID|     long|      lat|height|
+----------------+----------+---------+---------+------+
|       Bucuresti|         2|26.074412| 44.44677|     0|
|          Calais|         3| 1.811446|50.901549|     0|
|      Canterbury|         4| 1.075329|51.284212|     0|
|          Exeter|         5|-3.543547|50.729172|     0|
|Fideris, Bahnhof|         7| 9.733756|46.922368|   744|
+----------------+----------+---------+---------+------+
only showing top 5 rows



We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [8]:
metadata.count()

25935

In [9]:
# coordinates of Zürich main train station
lat_zurich = 47.3782
long_zurich = 8.5402

In [10]:
# convert to pandas dataframe
pandas_df = metadata.toPandas()
# keep only the stops that are located < 10km from Zurich HB
pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

In [11]:
# pandas_df.distance_to_zh.max()

In [63]:
# recreate spark dataframe from pandas dataframe
metadata = spark.createDataFrame(pandas_df)
# create dict of stations from pandas dataframe
stations = pandas_df.set_index('station_ID').to_dict('index')

### Cleaning main dataset

In [30]:
# load full data
# raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
raw_df = spark.read.load('/datasets/project/istdaten/2018/01', format='csv', header='true', inferSchema='true', sep=';')

In [31]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])

In [32]:
# refactor dates
df = df.withColumn('date', functions.from_unixtime(functions.unix_timestamp('date', 'dd.MM.yyyy')))
df = df.withColumn('schedule_arrival', functions.from_unixtime(functions.unix_timestamp('schedule_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_arrival', functions.from_unixtime(functions.unix_timestamp('real_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('schedule_dep', functions.from_unixtime(functions.unix_timestamp('schedule_dep', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_dep', functions.from_unixtime(functions.unix_timestamp('real_dep', 'dd.MM.yyyy HH:mm')))

In [33]:
# add a column containing the weekday (monday=1, sunday=6)
df = df.withColumn('weekday', helpers.get_weekday(df.date))

In [34]:
# keep only the rows with stops near zurich
df = df.where(df.stop_id.isin([int(x) for x in list(pandas_df.station_ID.unique())]))

In [35]:
# there is still 51'571'541 rows in zurich area
# df.count()

In [36]:
# keep only date after the 10th of december, because the schedule changed
df = df.where(df.date > '10.12.2017 00:00')

In [37]:
# discard the rows when there is no stop here
df2 = df.where(df.no_stop_here == 'false')

## Modeling the network

### From stops to trips

In [38]:
# create a column with the schedule time that will be used to build the network
df2 = df2.withColumn('schedule_time', helpers.date_choice(df2.schedule_arrival, df2.schedule_dep))
#df2 = df2.withColumn('schedule_time', functions.from_unixtime(functions.unix_timestamp('schedule_time', 'dd.MM.yyyy HH:mm')))

# create a column that tells if a stop is the first/last one of its trip or in the middle
df2 = df2.withColumn('stop_type', helpers.stop_type(df2.schedule_dep, df2.schedule_arrival))

In [39]:
trips = df2.select(['trip_id', 'date', 'schedule_time', 'stop_id', 'stop_type', 'schedule_arrival', 'schedule_dep']).orderBy(['trip_id', 'schedule_time'], ascending=[0,0])

In [40]:
# duplicate the dataframe, shift the copy of one row and append it to the original
# this way, we have for each row the current stop and the next stop
w = Window().partitionBy(functions.col('trip_id')).orderBy(functions.col('trip_id'))
trips2 = trips.select("*", functions.lag("trip_id").over(w).alias("next_tid"))
trips2 = trips2.select("*", functions.lag("schedule_time").over(w).alias("next_time"))
trips2 = trips2.select("*", functions.lag("stop_id").over(w).alias("next_sid"))
trips2 = trips2.select("*", functions.lag("stop_type").over(w).alias("next_type"))
trips2 = trips2.select("*", functions.lag("schedule_arrival").over(w).alias("next_sched_arr"))
trips2 = trips2.select("*", functions.lag("schedule_dep").over(w).alias("next_sched_dep"))

trips2 = trips2.where(trips2.next_time.isNotNull())

In [41]:
#trips2.where(trips2.stop_type=='first').count()
#trips2.where(trips2.stop_type=='last').count()
#trips2.where(trips2.stop_type=='mid').count()

In [48]:
# create a new column telling if the edge is valid or not
# (i.e. if the stop and next stop are really part of the same ride)
trips3 = trips2.withColumn('is_valid', helpers.edge_is_valid(trips2.trip_id, trips2.schedule_time, trips2.stop_id, trips2.stop_type, trips2.next_tid, trips2.next_time, trips2.next_sid, trips2.next_type))

In [49]:
# keep only valid edges
trips4 = trips3.filter(trips3.is_valid=='true')

In [50]:
# trips4.select('stop_id', 'next_sid').distinct().count()
# gives 6606

# trips3.select('stop_id', 'next_sid').distinct().count()
# gives 8557

### For each day of the week, model the network
Get the edges of the network and the departure/arrival times for each trip (edge=trip)
We assume the schedule repeat every week, and we generate one schedule per weekday.
Days off have the same schedules as sundays.

In [51]:
# creating a model for each day of the week
# this code needs to be run only once
typical_monday = '2018-01-15 00:00:00'
typical_tuesday = '2018-01-16 00:00:00'
typical_wednesday = '2018-01-17 00:00:00'
typical_thursday = '2018-01-18 00:00:00'
typical_friday = '2018-01-19 00:00:00'
typical_saturday = '2018-01-20 00:00:00'
typical_sunday = '2018-01-21 00:00:00'
typical_week = [typical_monday,typical_tuesday,typical_wednesday,typical_thursday,typical_friday,typical_saturday,typical_sunday]

In [92]:
regenerate_models = True
models = []
days_names = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

# generate one network for each weekday and store them in pickles
if regenerate_models:
    for (date, day_name) in zip(typical_week, days_names):
        network = helpers.model_network(trips4, date)
        with open('./data/'+day_name+'.pickle', 'wb') as handle:
            pickle.dump(network, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(str(day_name) + ' done')

monday done
tuesday done
wednesday done
thursday done
friday done
saturday done
sunday done


In [93]:
# load the networks from the pickles
for day in days_names:
    with open('./data/'+ day +'.pickle', 'rb') as handle:
        network = pickle.load(handle)
    models.append(network)

In [94]:
models[0][list(models[0].keys())[0]]

{8503001: [('2018-01-15 05:34:00', '2018-01-15 05:38:00'),
  ('2018-01-15 05:49:00', '2018-01-15 05:53:00'),
  ('2018-01-15 06:04:00', '2018-01-15 06:08:00'),
  ('2018-01-15 06:19:00', '2018-01-15 06:23:00'),
  ('2018-01-15 06:34:00', '2018-01-15 06:38:00'),
  ('2018-01-15 06:49:00', '2018-01-15 06:53:00'),
  ('2018-01-15 07:04:00', '2018-01-15 07:08:00'),
  ('2018-01-15 07:19:00', '2018-01-15 07:23:00'),
  ('2018-01-15 07:34:00', '2018-01-15 07:38:00'),
  ('2018-01-15 07:49:00', '2018-01-15 07:53:00'),
  ('2018-01-15 08:04:00', '2018-01-15 08:08:00'),
  ('2018-01-15 08:19:00', '2018-01-15 08:23:00'),
  ('2018-01-15 08:34:00', '2018-01-15 08:38:00'),
  ('2018-01-15 08:49:00', '2018-01-15 08:53:00'),
  ('2018-01-15 09:04:00', '2018-01-15 09:08:00'),
  ('2018-01-15 09:19:00', '2018-01-15 09:23:00'),
  ('2018-01-15 09:34:00', '2018-01-15 09:38:00'),
  ('2018-01-15 09:49:00', '2018-01-15 09:53:00'),
  ('2018-01-15 10:04:00', '2018-01-15 10:08:00'),
  ('2018-01-15 10:19:00', '2018-01-15 10: